# ConversationBufferWindowMemory

`ConversationBufferWindowMemory` 는 시간이 지남에 따라 대화의 상호작용 목록을 유지합니다.

이때, `ConversationBufferWindowMemory` 는 모든 대화내용을 활용하는 것이 아닌 **최근 K개** 의 상호작용만 사용합니다.

이는 버퍼가 너무 커지지 않도록 가장 최근 상호작용의 슬라이딩 창을 유지하는 데 유용할 수 있습니다.


In [2]:
# from langchain.memory import ConversationBufferWindowMemory

# memory = ConversationBufferWindowMemory(k=2, return_messages=True)

# memory.save_context(
#     inputs={
#         "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
#     },
#     outputs={
#         "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
#     },
# )
# memory.save_context(
#     inputs={"human": "네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"},
#     outputs={
#         "ai": "감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"},
#     outputs={
#         "ai": "업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"},
#     outputs={
#         "ai": "본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"},
#     outputs={
#         "ai": "입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."
#     },
# )
# memory.save_context(
#     inputs={"human": "모든 절차를 완료했습니다. 계좌가 개설된 건가요?"},
#     outputs={
#         "ai": "네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!"
#     },
# )


from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

K = 2
# memory = ConversationBufferWindowMemory(k=2, return_messages=True) 이거 대체
# 1. LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


# 2. Window Memory 설정 (k=2)
history = InMemoryChatMessageHistory()

def add_with_window(history_obj, message):
    history_obj.add_message(message)
    if len(history_obj.messages) > K * 2:  # K 쌍의 메시지 유지
        history_obj.messages = history_obj.messages[-K * 2 :] # 가장 최근 k개

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | llm # base chain ( 프롬프트 어뎁터 ) # dict -> str 로 바꿔서 llm 에게 전달 

conversation = RunnableWithMessageHistory(
    chain,
    lambda session_id: history,
    input_messages_key="messages",
    history_messages_key="messages"
)

# 3. 기존 대화 복원 (최신 방식)
messages = [
    HumanMessage("안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"),
    AIMessage("안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"),

    HumanMessage("네, 신분증을 준비했습니다. 이제 무엇을 해야 하나요?"),
    AIMessage("감사합니다. 신분증 앞뒤를 명확하게 촬영하여 업로드해 주세요. 이후 본인 인증 절차를 진행하겠습니다."),

    HumanMessage("사진을 업로드했습니다. 본인 인증은 어떻게 진행되나요?"),
    AIMessage("업로드해 주신 사진을 확인했습니다. 이제 휴대폰을 통한 본인 인증을 진행해 주세요. 문자로 발송된 인증번호를 입력해 주시면 됩니다."),

    HumanMessage("인증번호를 입력했습니다. 계좌 개설은 이제 어떻게 하나요?"),
    AIMessage("본인 인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택하고 필요한 정보를 입력해 주세요. 예금 종류, 통화 종류 등을 선택할 수 있습니다."),

    HumanMessage("정보를 모두 입력했습니다. 다음 단계는 무엇인가요?"),
    AIMessage("입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요."),

    HumanMessage("모든 절차를 완료했습니다. 계좌가 개설된 건가요?"),
    AIMessage("네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!")
]

# 4. 메모리에 대화 직접 push
for msg in messages:
    add_with_window(history, msg)

print(history.messages)
# # 5. 이제 호출 가능
# session_id_key = "history"

# response = conversation.invoke({
#     "messages": [HumanMessage("추가로 체크카드도 발급할 수 있나요?")]
    
#     },
#  config={"configurable": {"session_id": session_id_key}}
#  )
#print(response)
#print(response.content)


c:\Users\jeffr\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-SGRq4Tjy-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[HumanMessage(content='정보를 모두 입력했습니다. 다음 단계는 무엇인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='모든 절차를 완료했습니다. 계좌가 개설된 건가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!', additional_kwargs={}, response_metadata={})]


대화기록을 확인해 보면 **최근 2개** 의 메시지만 반환하는 것을 확인할 수 있습니다.


In [3]:
# 대화 기록을 확인합니다.
# memory.load_memory_variables({})["history"]

print(history.messages)


[HumanMessage(content='정보를 모두 입력했습니다. 다음 단계는 무엇인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='입력해 주신 정보를 확인했습니다. 계좌 개설 절차가 거의 끝났습니다. 마지막으로 이용 약관에 동의해 주시고, 계좌 개설을 최종 확인해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='모든 절차를 완료했습니다. 계좌가 개설된 건가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 계좌 개설이 완료되었습니다. 고객님의 계좌 번호와 관련 정보는 등록하신 이메일로 발송되었습니다. 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 감사합니다!', additional_kwargs={}, response_metadata={})]
